## Notebook 2: Apprentissage automatique, Regression: Polution CO2

#### Importation des librairies

In [64]:
import pandas as pd
import numpy as np
import streamlit as st
#-------------------------------------------------
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#-------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor, make_column_transformer

#--------------------------------------------------------
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

#--------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance
#------------------------------------------------
from xgboost import XGBRegressor

#-----------------------------------------------------------
from sklearn.model_selection import GridSearchCV

## Checklist

Depuis 2001, **l’ADEME** acquiert tous les ans ces données auprès de **l’Union Technique de l’Automobile du motocycle et du Cycle UTAC** (en charge de l’homologation des véhicules avant leur mise en vente) en accord avec le ministère du développement durable.
Pour chaque véhicule les données d’origine (transmises par l’Utac) sont les suivantes :

* **Les consommations de carburant**

* **Les émissions de dioxyde de carbone (CO2)**

* **Les émissions des polluants de l’air** (réglementés dans le cadre de la norme Euro)

* **L’ensemble des caractéristiques techniques des véhicules** (gammes, marques, modèles, n° de CNIT, type d’énergie ...)



# L'inventaire des varaibles pertinentes:

Les données comprenent des variables pertinentes suivantes:

* **lib_mrq_utac**: La marque, il y'a 12.

* **lib_mod**: Le modèle commerciale, il y'a 20.

* **cod_cbr**: Le type de carburant, il y a 5.

* **hybride**: Information permettant d’identifier les véhicules hybrides (O/N)

* **puiss_max** : Puissance maximale

* **typ_boite_nb_rapp**: Type boite de vitesse et le nombre de rapport.

* **conso_urb**: Consommation urbaine de carburant (en l/100km),

* **conso_exurb**: consommation extra urbaine de carburant (en l/100km),

* **conso_mixte**: Consommation mixte de carburant (en l/100km),

* **co2**: Emission de CO2 (en g/km),

* **masse_ordma_min**: Masse en ordre de marche mini

* **masse_ordma_max**: Masse en ordre de marche max
 
* **Carrosserie**: Carrosserie

* **gamme**: Gamme du véhicule



# Objectif

Notre objectif majeur dans ce notebook 2 est de :

Prédire les emisisions de **CO2** des vehicules en fonction de certaines informations (Variables explicatives)

* En utilisant 4 à 5 modéles différents

* En comparant  les scores
    
* En choissisant le meilleur modèle


# Description des données

Lien vers les données: https://www.data.gouv.fr/fr/datasets/emissions-de-co2-et-de-polluants-des-vehicules-commercialises-en-france/


### Chargement, lecture, apercu et infos des données

In [65]:
data_model = pd.read_csv("data_model.csv")

In [66]:
encoder = LabelEncoder()
data_model["Carrosserie"] = encoder.fit_transform(data_model["Carrosserie"])

### Selectionner les feautures les plus importans

In [67]:
New_Data = data_model[['Carrosserie', 'masse_ordma_min', 'masse_ordma_max', 'co2']]

In [68]:
y = New_Data['co2']
X = New_Data.drop(['co2'] ,axis =1)

### Traiter la colonne Carrosserie

### Prediction de CO2
    
Pour chacun de nos modèles:
   * **DummyRegressor**,
   * **LinearRegression**, 
   * **SGDRegressor**, 
   * **RandomForestRegressor**,
   * **GradientBoostingRegressor**,
   * **XGBRegressor**, 
    
Nous allons predire le niveau d'émission de **Co2** puis établir une comparaison entre les performances de chaque modèle.

    Appliquer un GridSearch pour optimiser les hyperparamètres de chaque modèle
    Paramètrer dans GridSearch les scores qui sont adaptés à la régression (R2, MAE et RMSE)
    Identifier le meilleur modèle

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

classifiers = [DummyRegressor(), LinearRegression(), SGDRegressor(), RandomForestRegressor(), GradientBoostingRegressor(), XGBRegressor()]

param_grids = [{'strategy': ["mean", "median"]},
               {"fit_intercept": [True, False]},
               {"max_iter" : [1000, 900, 1100 ]},
               {"n_estimators" : [50]},
               {"n_estimators" : [50]},
               {"n_estimators" : [50]}]

# Create a list to store the results
results = []

# Loop through the classifiers and their hyperparameter grids
for i, (clf, param_grid) in enumerate(zip(classifiers, param_grids)):
    # Create the GridSearchCV object
    grid_search = GridSearchCV(clf, param_grid, scoring='r2' , refit='r2')
    
    # Fit the GridSearchCV object to the data
    grid_search.fit(X, y)
    
    # Store the results
    results.append((type(clf).__name__, grid_search.best_params_, grid_search.best_score_))
    
    # Print the best parameters and scores
    print("=============================================================")
    print(f"Classifier {i+1}: {type(clf).__name__}")
    print(f"best parameters: {grid_search.best_params_}")
    print(f"best score: {grid_search.best_score_}")
    print("=============================================================\n")

# Find the winner
winner = max(results, key=lambda x: x[2])

# Print the winner
print("=============================================================")
print(f"The winner is {winner[0]} with a score of {winner[2]} and parameters:")
print(winner[1])
print("=============================================================")

Classifier 1: DummyRegressor
best parameters: {'strategy': 'median'}
best score: -0.21342788285930653

Classifier 2: LinearRegression
best parameters: {'fit_intercept': True}
best score: 0.15675550225255272



Classifier 3: SGDRegressor
best parameters: {'max_iter': 1100}
best score: -5.779604858500127e+26

Classifier 4: RandomForestRegressor
best parameters: {'n_estimators': 50}
best score: 0.24345933774717085

Classifier 5: GradientBoostingRegressor
best parameters: {'n_estimators': 50}
best score: 0.28544965565149766

Classifier 6: XGBRegressor
best parameters: {'n_estimators': 50}
best score: 0.30894490738898794

The winner is XGBRegressor with a score of 0.30894490738898794 and parameters:
{'n_estimators': 50}


### Application Web

Développer une application Streamlit avec les options suivantes :

    1- L'utilisateur doit selectionner le type du carrosserie à partire d'un menu déroulant
    2- L'utilisateur doit saisir 'masse_ordma_min' et 'masse_ordma_max' dans deux champs de saisie différent
    3- Programmer un boutton pour lancer la prédiction de CO2

In [70]:
import joblib

# Entraînement du modèle (votre code d'entraînement ici)
best_model = GradientBoostingRegressor(n_estimators=50)
best_model.fit(X, y)

# Sauvegarde du modèle dans un fichier
joblib.dump(best_model, 'meilleur_modele.pkl')

['meilleur_modele.pkl']